In [1]:
import torch

dc_model = torch.load('/vulcanscratch/mgwillia/unsupervised-classification/backbones/dcv2_r50_800.pth.tar', map_location='cpu')
dc_projections = torch.load('/vulcanscratch/mgwillia/vissl/features/dcv2_r50_800_imagenet_projections.pth.tar', map_location='cpu')
dc_train_projections = dc_projections['train_features']
dc_val_projections = dc_projections['val_features']

dc_protoype_1 = dc_model['prototypes.prototypes0.weight']
dc_protoype_2 = dc_model['prototypes.prototypes1.weight']
dc_protoype_3 = dc_model['prototypes.prototypes2.weight']

swav_model = torch.load('/vulcanscratch/mgwillia/unsupervised-classification/backbones/swav_r50_800.torch', map_location='cpu')
swav_projections = torch.load('/vulcanscratch/mgwillia/vissl/features/swav_r50_800_imagenet_projections.pth.tar', map_location='cpu')
swav_train_projections = swav_projections['train_features']
swav_val_projections = swav_projections['val_features']

swav_prototype = swav_model['classy_state_dict']['base_model']['model']['heads']['0.prototypes0.weight']

print(dc_train_projections.shape)
print(dc_val_projections.shape)
print(swav_train_projections.shape)
print(swav_val_projections.shape)

print(dc_protoype_1.shape)
print(dc_protoype_2.shape)
print(dc_protoype_3.shape)
print(swav_prototype.shape)

cluster_train_codes_list = []
cluster_val_codes_list = []
for dc_prototype in [dc_protoype_1, dc_protoype_2, dc_protoype_3]:
    cluster_train_codes = torch.mm(dc_train_projections, dc_prototype.t())
    cluster_val_codes = torch.mm(dc_val_projections, dc_prototype.t())
    print(cluster_train_codes.shape)
    print(cluster_val_codes.shape)
    cluster_train_codes_list.append(cluster_train_codes)
    cluster_val_codes_list.append(cluster_val_codes)
cluster_train_codes = torch.mm(swav_train_projections, swav_prototype.t())
cluster_val_codes = torch.mm(swav_val_projections, swav_prototype.t())
cluster_train_codes_list.append(cluster_train_codes)
cluster_val_codes_list.append(cluster_val_codes)

torch.Size([1281167, 128])
torch.Size([50000, 128])
torch.Size([1281167, 128])
torch.Size([50000, 128])
torch.Size([3000, 128])
torch.Size([3000, 128])
torch.Size([3000, 128])
torch.Size([3000, 128])
torch.Size([1281167, 3000])
torch.Size([50000, 3000])
torch.Size([1281167, 3000])
torch.Size([50000, 3000])
torch.Size([1281167, 3000])
torch.Size([50000, 3000])


In [2]:
import numpy as np
import torch

train_targets = torch.load(f'/vulcanscratch/mgwillia/vissl/features/swav_r50_800_imagenet_targets.pth.tar')['train_targets'].numpy()
val_targets = torch.load(f'/vulcanscratch/mgwillia/vissl/features/swav_r50_800_imagenet_targets.pth.tar')['val_targets'].numpy()

for i, train_codes in enumerate(cluster_train_codes_list):
    val_codes = cluster_val_codes_list[i]
    train_clusters = torch.argmax(train_codes, dim=1)
    val_clusters = torch.argmax(val_codes, dim=1)
    cluster_target_votes = torch.zeros((train_codes.shape[1], 1000))
    for j in range(train_clusters.shape[0]):
        cur_cluster = train_clusters[j]
        cur_target = train_targets[j]
        cluster_target_votes[cur_cluster][cur_target] += 1
    overcluster_label_mapping = torch.argmax(cluster_target_votes, dim=1).numpy()
    print(overcluster_label_mapping.shape)
    remapped_predictions = np.zeros(val_clusters.shape[0])
    for j in range(val_clusters.shape[0]):
        remapped_predictions[j] = overcluster_label_mapping[val_clusters[j]]
    val_accuracy =  np.where((val_targets == remapped_predictions), 1, 0).mean()
    print(f'Validation accuracy: {val_accuracy}')

(3000,)
Validation accuracy: 0.54316
(3000,)
Validation accuracy: 0.54256
(3000,)
Validation accuracy: 0.54478
(3000,)
Validation accuracy: 0.489


In [3]:
import numpy as np
import os
import torch

train_targets = torch.load(f'/vulcanscratch/mgwillia/vissl/features/swav_r50_800_imagenet_targets.pth.tar')['train_targets'].numpy()
val_targets = torch.load(f'/vulcanscratch/mgwillia/vissl/features/swav_r50_800_imagenet_targets.pth.tar')['val_targets'].numpy()

root = '/vulcanscratch/mgwillia/vissl/clusters/'
methods = ['btwins_r50_1000', 'dcv2_r50_800', 'moco_r50_800']
for method in methods:
    train_clusters = np.load(f'/vulcanscratch/mgwillia/vissl/clusters/{method}_imagenet3k_labels.npy')
    val_clusters = np.load(f'/vulcanscratch/mgwillia/vissl/clusters/{method}_imagenet3k_predictions.npy')
    cluster_target_votes = np.zeros((train_clusters.shape[0], 1000))
    for i in range(train_clusters.shape[0]):
        cur_cluster = train_clusters[i]
        cur_target = train_targets[i]
        cluster_target_votes[cur_cluster][cur_target] += 1
    overcluster_label_mapping = np.argmax(cluster_target_votes, axis=1)
    print(overcluster_label_mapping.shape)
    remapped_predictions = np.zeros(val_clusters.shape[0])
    for i in range(val_clusters.shape[0]):
        remapped_predictions[i] = overcluster_label_mapping[val_clusters[i]]
    val_accuracy = np.where((val_targets == remapped_predictions), 1, 0).mean()
    print(f'Validation accuracy: {val_accuracy}')
    

(1281167,)
Validation accuracy: 0.4215
Validation accuracy: 0.4215
(1281167,)
Validation accuracy: 0.43024
Validation accuracy: 0.43024
(1281167,)
Validation accuracy: 0.43672
Validation accuracy: 0.43672
